# Subscriptions API & Planet SDK Quickstart


Getting started with Planet SDK and the Subscriptions API.

The Subscriptions API is Planet's next-gen data delivery API. With a single API call, the Subscriptions API allows you to subscribe to continuous cloud delivery of imagery and metadata collections, which meet your item filter criteria. In this notebook, we'll take a look at this functionality and how you might be able to incorporate it into your own workflows.

## Planet API Authentication & Cloud Delivery Credentials
The new SDK supports a couple different methods of authentication (and soon, will support OIDC / Okta token-based authentication as well). Read more in the docs here.

For this example, we'll use the getpass tool to pass in our API key as a string to the from_key method.

In this example, we will also be using Gogle Cloud Storage as our cloud delivery service. However, it is also possible to use: <br> <br> - Amazon S3 <br>  - Oracle Cloud Storage <br>  - Microsoft Azure <br> <br> Regardless of the platform you're using, ensure that you have both write and delete access to your storage location. <br> For how details on how to change the delivery schema when using each of these cloud delivery platforms, see the [Subscriptions API Delivery Documentation](https://developers.planet.com/docs/subscriptions/delivery/) on our Developer's Center.

In [372]:
import datetime

In [ ]:
from getpass import getpass
from planet import Auth
api_key = getpass('Enter your API key:')
bucket_name = "GCS_BUCKET_NAME_HERE"
gcs_key = "GCS_API_KEY_HERE"

auth = Auth.from_key(api_key)

## Listing a Subscription

First, let's try listing any existing subscriptions we already have in our Planet accounts. If you've never made a subscription before, this may show up as an empty list. <br> <br> We'll try this again below after we've created our first subscription.

In [ ]:
from planet import Session
from planet.clients.subscriptions import SubscriptionsClient

In [ ]:
# We want to initiate and authenticate into the Client, and then list up to 5 of our recent running or pending subscriptions
async with Session(auth=auth) as sess:
    cl = SubscriptionsClient(sess)
    subscriptions = cl.list_subscriptions(limit=5, status=['running', 'pending', 'completed'])
    sub_list = [i async for i in subscriptions]

In [ ]:
# Note: if this shows an empty list "[]", this may mean that you don't have any currently running or pending subscriptions!
sub_list

## Creating a Subscription

In order to create a subscription using the Client, we must provide a subscription description dictionary, including a `name`, and `catalog` source block (with item_types, asset_types, geometry, start_time, end_time), along with a `delivery` block with a specified cloud storage location. Subscriptions also support select tools.

The `rrule` attribute leverages [iCalendar recurrence rules](https://icalendar.org/iCalendar-RFC-5545/3-8-5-3-recurrence-rule.html). This can be used to create subscriptions that deliver items for recurring periods within the total coverage time. In this example, we are getting monthly data between January to November, from 2022 to 2025. More information on how to use this attribute can be found [here](https://developers.planet.com/docs/subscriptions/source/#rrules-recurrence-rules).

Feel free to use the below request format as a template to input the parameters you need for your own subscription.

In [546]:
sub_desc = {
    "name": "Recurring Subscription",
    "source": {
        "type": "catalog",
        "parameters": {
            "geometry": {
                "coordinates": [[[139.5648193359375,35.42374884923695],
                                [140.1031494140625,35.42374884923695],
                                [140.1031494140625,35.77102915686019],
                                [139.5648193359375,35.77102915686019],
                                [139.5648193359375,35.42374884923695]]],
                "type": "Polygon"
            },
            "start_time": "2022-01-01T00:00:00Z",
            "end_time": "2025-11-01T00:00:00Z",
            "rrule": "FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10",
            "item_types": ["PSScene"],
            "asset_types": ["ortho_analytic_4b"]
        }
    },
    "delivery": {"type": "google_cloud_storage",
                "parameters": {
                    "bucket":bucket_name,
         "credentials":gcs_key}
                }
}

In [ ]:
sub_desc

In [548]:
# Now we can create a corresponding subscription using our description
async with Session(auth=auth) as sess:
    cl = SubscriptionsClient(sess)
    subscription_desc = await cl.create_subscription(sub_desc)

In [549]:
subscription_desc

{'name': 'Recurring Subscription',
 'source': {'type': 'catalog',
  'parameters': {'asset_types': ['ortho_analytic_4b'],
   'end_time': '2025-11-01T00:00:00Z',
   'geometry': {'coordinates': [[[139.5648193359375, 35.42374884923695],
      [140.1031494140625, 35.42374884923695],
      [140.1031494140625, 35.77102915686019],
      [139.5648193359375, 35.77102915686019],
      [139.5648193359375, 35.42374884923695]]],
    'type': 'Polygon'},
   'item_types': ['PSScene'],
   'rrule': 'FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10',
   'start_time': '2022-01-01T00:00:00Z'}},
 'delivery': {'type': 'google_cloud_storage',
  'parameters': {'bucket': 'jub-orders-delivery',
   'credentials': '<REDACTED>'}},
 'created': '2022-11-21T16:09:00.250108Z',
 '_links': {'_self': 'https://api.planet.com/subscriptions/v1/2607a8b6-d4a9-4b76-85b9-6de967c5f8e0'},
 'status': 'preparing',
 'id': '2607a8b6-d4a9-4b76-85b9-6de967c5f8e0',
 'updated': '2022-11-21T16:09:00.250108Z'}

## Listing a Subscription

Let's try listing our 5 most recent existing subscriptions we have associated with our Planet account. We should see the subscription we just created above as part of this.

You can filter the subscriptions listed by status- `preparing`, `running`, `pending`, `completed`, or `canceled`.

For this example, we will only list the subscriptions we have that have not been canceled.

In [550]:
# We want to initiate and authenticate into the Client, and then list up to 5 of our recent subscriptions
async with Session(auth=auth) as sess:
    cl = SubscriptionsClient(sess)
    subscriptions = cl.list_subscriptions(limit=5, status=['preparing', 'running', 'pending', 'completed'])
    sub_list = [i async for i in subscriptions]

In [551]:
# You should see at least the subscription we created above as part of this list!
sub_list

[{'name': 'Recurring Subscription',
  'source': {'type': 'catalog',
   'parameters': {'asset_types': ['ortho_analytic_4b'],
    'end_time': '2025-11-01T00:00:00Z',
    'geometry': {'coordinates': [[[139.5648193359375, 35.42374884923695],
       [140.1031494140625, 35.42374884923695],
       [140.1031494140625, 35.77102915686019],
       [139.5648193359375, 35.77102915686019],
       [139.5648193359375, 35.42374884923695]]],
     'type': 'Polygon'},
    'item_types': ['PSScene'],
    'rrule': 'FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10',
    'start_time': '2022-01-01T00:00:00Z'}},
  'delivery': {'type': 'google_cloud_storage',
   'parameters': {'bucket': 'jub-orders-delivery',
    'credentials': '<REDACTED>'}},
  'created': '2022-11-21T16:09:00.250108Z',
  '_links': {'_self': 'https://api.planet.com/subscriptions/v1/2607a8b6-d4a9-4b76-85b9-6de967c5f8e0'},
  'status': 'preparing',
  'id': '2607a8b6-d4a9-4b76-85b9-6de967c5f8e0',
  'updated': '2022-11-21T16:09:00.250108Z'}]

In [552]:
# Let's save the subscription ID for our first subscription. 
# We will need this in order to retrieve information about our subscription in the next "Get Subscription" step.
sub_id = sub_list[0]["id"]

## Get a Subscription

We can use our subscription ID for a given subscription to retrieve the corresponding subscription's description. Let's see this in action using the subscription ID we saved above.

In [553]:
# We want to initiate and authenticate into the Client, and then get our subscription description matching the ID
async with Session(auth=auth) as sess:
    cl = SubscriptionsClient(sess)
    subscription_desc = await cl.get_subscription(subscription_id=sub_id)

In [554]:
subscription_desc

{'name': 'Recurring Subscription',
 'source': {'type': 'catalog',
  'parameters': {'asset_types': ['ortho_analytic_4b'],
   'end_time': '2025-11-01T00:00:00Z',
   'geometry': {'coordinates': [[[139.5648193359375, 35.42374884923695],
      [140.1031494140625, 35.42374884923695],
      [140.1031494140625, 35.77102915686019],
      [139.5648193359375, 35.77102915686019],
      [139.5648193359375, 35.42374884923695]]],
    'type': 'Polygon'},
   'item_types': ['PSScene'],
   'rrule': 'FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10',
   'start_time': '2022-01-01T00:00:00Z'}},
 'delivery': {'type': 'google_cloud_storage',
  'parameters': {'bucket': 'jub-orders-delivery',
   'credentials': '<REDACTED>'}},
 'created': '2022-11-21T16:09:00.250108Z',
 '_links': {'_self': 'https://api.planet.com/subscriptions/v1/2607a8b6-d4a9-4b76-85b9-6de967c5f8e0'},
 'status': 'running',
 'id': '2607a8b6-d4a9-4b76-85b9-6de967c5f8e0',
 'updated': '2022-11-21T16:09:08.179889Z'}

## Update a Subscription

What if we see something about our subscription that we want to change? We can update a subscription by specifying the subscription ID whose attributes we want to alter, along with a new request with the updated parameters. 

**Note**: After a subscription transitions to `running`, changes to the `start_time` and `item_types` fields are not allowed. The edit will only apply to future item publications and deliveries and no items will be redelivered. 
Backfill subscriptions cannot be edited. If you need to reorder archive items based on an updated filter or tool specifications, you can search for and order archive items with the Data API and Orders API.

In [ ]:
# Let's look at our original request once more:
sub_desc

Let's change our end time January 1st, change our end year to 2027, and only look at images every other month.

In [556]:
sub_desc1 = {
    "name": "Recurring Subscription 1",
    "source": {
        "type": "catalog",
        "parameters": {
            "geometry": {
                "coordinates": [[[139.5648193359375,35.42374884923695],
                                [140.1031494140625,35.42374884923695],
                                [140.1031494140625,35.77102915686019],
                                [139.5648193359375,35.77102915686019],
                                [139.5648193359375,35.42374884923695]]],
                "type": "Polygon"
            },
            "start_time": "2022-01-01T00:00:00Z",
            "end_time": "2027-01-01T00:00:00Z",
            "rrule": "FREQ=MONTHLY;BYMONTH=2,4,6,8,10,12",
            "item_types": ["PSScene"],
            "asset_types": ["ortho_analytic_4b"]
        }
    },
    "delivery": {"type": "google_cloud_storage",
                "parameters": {
                    "bucket":bucket_name,
         "credentials":gcs_key}
                }
}

Now, we can update our subscription with this new request:

In [557]:
async with Session(auth=auth) as sess:
    cl = SubscriptionsClient(sess)
    subscription_desc1 = await cl.update_subscription(subscription_id=sub_id, request = sub_desc1)

Check out the new parameters of our subscription below. Notice that the frequency and end dates of the subscription have changed! Also note that this is still the same subscription ID as we were working with above.

In [558]:
subscription_desc1

{'name': 'Recurring Subscription 1',
 'source': {'type': 'catalog',
  'parameters': {'asset_types': ['ortho_analytic_4b'],
   'end_time': '2027-01-01T00:00:00Z',
   'geometry': {'coordinates': [[[139.5648193359375, 35.42374884923695],
      [140.1031494140625, 35.42374884923695],
      [140.1031494140625, 35.77102915686019],
      [139.5648193359375, 35.77102915686019],
      [139.5648193359375, 35.42374884923695]]],
    'type': 'Polygon'},
   'item_types': ['PSScene'],
   'rrule': 'FREQ=MONTHLY;BYMONTH=2,4,6,8,10,12',
   'start_time': '2022-01-01T00:00:00Z'}},
 'delivery': {'type': 'google_cloud_storage',
  'parameters': {'bucket': 'jub-orders-delivery',
   'credentials': '<REDACTED>'}},
 'created': '2022-11-21T16:09:00.250108Z',
 '_links': {'_self': 'https://api.planet.com/subscriptions/v1/2607a8b6-d4a9-4b76-85b9-6de967c5f8e0'},
 'status': 'running',
 'id': '2607a8b6-d4a9-4b76-85b9-6de967c5f8e0',
 'updated': '2022-11-21T16:09:27.805202Z'}

## Get Subscription Results

Now that we've made sure our subscription description matches what we want, we can see our subscription results using the `get_results` Client method.

In [559]:
# We want to initiate and authenticate into the Client, and then get our subscription results description matching the ID
async with Session(auth=auth) as sess:
    cl = SubscriptionsClient(sess)
    results_desc = cl.get_results(subscription_id=sub_id, limit=5)
    results_desc_list = [i async for i in results_desc]

In [560]:
results_desc

<async_generator object SubscriptionsClient.get_results at 0x7fa2a8d833a0>

Check out the results description below. Note that we can see the subscription created and updated date, as well as the status. 

Note that this step will only work for subscriptions that are currently running. 
If you get an empty list, it may mean that your subscription is still pending, especially if the start date of the subscription has not arrived yet.

In [561]:
results_desc_list

[{'id': '6c13f808-192a-4d81-9f8b-0f640633ac4d',
  'status': 'queued',
  'properties': {'item_id': '20220410_010050_90_248b',
   'item_types': ['PSScene']},
  'created': '2022-11-21T16:09:12.900168Z',
  'updated': '2022-11-21T16:09:28.326435Z',
  'errors': {},
  'outputs': []},
 {'id': '36c6ad46-2481-468e-8df6-30409663660a',
  'status': 'queued',
  'properties': {'item_id': '20220410_010055_51_248b',
   'item_types': ['PSScene']},
  'created': '2022-11-21T16:09:12.899517Z',
  'updated': '2022-11-21T16:09:28.398142Z',
  'errors': {},
  'outputs': []},
 {'id': 'ddec7fc7-6a6d-458b-b512-e5f94d6c6487',
  'status': 'queued',
  'properties': {'item_id': '20220409_002847_38_2455',
   'item_types': ['PSScene']},
  'created': '2022-11-21T16:09:12.899373Z',
  'updated': '2022-11-21T16:09:28.396796Z',
  'errors': {},
  'outputs': []},
 {'id': 'ea5df5b4-f27b-4d29-be18-ec02e8f34242',
  'status': 'queued',
  'properties': {'item_id': '20220409_002845_09_2455',
   'item_types': ['PSScene']},
  'created

## Cancel A Subscription

We can cancel a subscription by using the `cancel_subscription` method and providing the subscription_id corresponding to the subscription we want to cancel.

In [562]:
async with Session(auth=auth) as sess:
    cl = SubscriptionsClient(sess)
    cancel_sub = await cl.cancel_subscription(subscription_id=sub_id)

We can confirm this subscription is canceled by getting a description of this subscription using this ID. We should its status reported as 'canceled'.

In [563]:
async with Session(auth=auth) as sess:
    cl = SubscriptionsClient(sess)
    subscription_desc = await cl.get_subscription(subscription_id=sub_id)

In [564]:
subscription_desc['status']

'cancelled'

## Congratulations!

You now have all the tools you need to create and edit your own subscriptions using our Subscriptions API Client tool. For more information on our subscriptions API, please check out our [Subscriptions API  Documentation](https://developers.planet.com/docs/subscriptions/)